# Carga de audios, transcripción, limpieza y almacenamiento

In [6]:
import os
#import mysql.connector
import re

# Install whisper package
#%pip install openai-whisper

import whisper
import subprocess

In [7]:
# Folderes donde se ubican los audios y donde se almacenan los txt
audio_folder = "C:/Users/Usuario/tfm_call_optimizer/audios/"
transcription_folder = "C:/Users/Usuario/tfm_call_optimizer/transcripciones"

# Connect to MySQL
conn = mysql.connector.connect(
    host="your-mysql-server",
    user="your-username",
    password="your-password",
    database="your-database"
)
cursor = conn.cursor()

In [8]:
# Asegurémonos de que `ffmpeg` esté en el PATH antes de la transcripción
ffmpeg_path = r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe"

# Verificar si ffmpeg está accesible desde Python
try:
    subprocess.run([ffmpeg_path, "-version"], check=True, capture_output=True)
    print("✅ ffmpeg está correctamente accesible.")
except FileNotFoundError:
    print("❌ No se pudo encontrar ffmpeg en la ruta especificada.")
    exit()


✅ ffmpeg está correctamente accesible.


In [22]:
filename = "3_nodasolucion_vivi.wav"  # Nombre del archivo de prueba con extensión
audio_path = os.path.join(audio_folder, filename)
transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt"))

print(f"Buscando archivo en: {audio_path}")
if os.path.exists(audio_path):
    print("✅ El archivo existe.")
else:
    print("❌ El archivo NO existe. Revisa la ruta.")

# Transcribir audio
#result = model.transcribe(audio_path,shell=True)
#raw_text = result["text"]


Buscando archivo en: C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi.wav
✅ El archivo existe.


In [11]:
# Verificar si ffmpeg está accesible desde Python
try:
    subprocess.run([ffmpeg_path, "-version"], check=True, capture_output=True)
    print("✅ ffmpeg está correctamente accesible.")
except FileNotFoundError:
    print("❌ No se pudo encontrar ffmpeg en la ruta especificada.")
    exit()

✅ ffmpeg está correctamente accesible.


In [12]:
# Asegurémonos de que `ffmpeg` esté en el PATH antes de la transcripción
ffmpeg_path = r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe"

In [15]:
# Configurar ffmpeg manualmente en whisper
whisper.utils.ffmpeg_bin = ffmpeg_path  # Esto le dice a whisper dónde encontrar ffmpeg

In [16]:
# Load Whisper model
model = whisper.load_model("medium")

In [25]:
# Ruta del archivo WAV
input_audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi.WAV"
output_audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi_converted.mp3"

# Comando para convertir WAV a MP3
ffmpeg_command = [
    r"C:\Users\Usuario\ffmpeg-2025-03-06-git-696ea1c223-essentials_build\bin\ffmpeg.exe",
    "-i", input_audio_path,
    output_audio_path
]

# Ejecutar el comando
try:
    subprocess.run(ffmpeg_command, check=True)
    print(f"✅ El archivo se ha convertido exitosamente a: {output_audio_path}")
except subprocess.CalledProcessError as e:
    print(f"❌ Error en la conversión: {e}")

# Ahora, usa este archivo convertido para transcribirlo con whisper
audio_path = output_audio_path

✅ El archivo se ha convertido exitosamente a: C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi_converted.mp3


In [26]:
# Transcribir el archivo de audio convertido
audio_path = "C:/Users/Usuario/tfm_call_optimizer/audios/3_nodasolucion_vivi_converted.mp3"

# Transcribir audio
result = model.transcribe(audio_path)

# Obtener el texto transcrito
raw_text = result["text"]

# Imprimir el texto transcrito
print(f"Texto transcrito: {raw_text}")

# Guardar la transcripción en un archivo de texto
transcription_path = "C:/Users/Usuario/tfm_call_optimizer/transcripciones/3_nodasolucion_vivi_converted.txt"
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(raw_text)

print(f"✅ Transcripción guardada en: {transcription_path}")

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [23]:
# Transcribir audio
print("Transcribiendo audio...")
result = model.transcribe(audio_path)  # Si la ruta es correcta, no se necesita shell=True


Transcribiendo audio...


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [ ]:
# Limpiar transcripción
cleaned_text = clean_transcription(raw_text)

# Guardar en archivo de texto
with open(transcription_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"✅ Saved cleaned transcription: {transcription_path}")


# Function to clean transcription text
def clean_transcription(text):
    """
    Cleans the transcription text by removing filler words, extra spaces,
    and unwanted artifacts.
    """
    # Define common filler words (can be extended)
    filler_words = ["eh", "mmm", "este", "o sea", "pues", "bueno", "entonces", "verdad", "sabes", "digo"]
    filler_pattern = r'\b(?:' + '|'.join(filler_words) + r')\b'

    # Remove filler words
    text = re.sub(filler_pattern, '', text, flags=re.IGNORECASE)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Remove unnecessary punctuation repetition (e.g., "Hello...." → "Hello.")
    text = re.sub(r'([.?!])\1+', r'\1', text)

    # Strip leading/trailing spaces
    text = text.strip()

    return text



for filename in os.listdir(audio_folder):
    if filename.endswith(".wav") or filename.endswith(".mp3"):
        audio_path = os.path.join(audio_folder, filename)
        transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt").replace(".mp3", ".txt"))

        # Transcribe audio
        result = model.transcribe(audio_path)
        raw_text = result["text"]

        # Clean transcription text
        cleaned_text = clean_transcription(raw_text)

        # Save cleaned transcription to a .txt file
        with open(transcription_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)

        # Insert cleaned transcription into MySQL
        sql = """
        INSERT INTO transcriptions (call_id, transcription, status)
        VALUES (%s, %s, %s)
        """
        values = (filename.replace(".wav", "").replace(".mp3", ""), cleaned_text, "Pending")

        cursor.execute(sql, values)
        conn.commit()

        print(f"✅ Saved cleaned transcription: {transcription_path} & stored in database.")

cursor.close()
conn.close()
